In [1]:
!pip install --quiet duckdb
!pip install --quiet jupysql 
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib

In [2]:
import duckdb
import pandas as pd
# No need to import sqlalchemy or duckdb_engine
#  JupySQL will use SQLAlchemy to auto-detect the driver needed based on your connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [5]:
%%sql
SELECT
    schema_name,
    function_name
FROM duckdb_functions()
ORDER BY ALL DESC
LIMIT 5

,schema_name,function_name
0,pg_catalog,shobj_description
1,pg_catalog,shobj_description
2,pg_catalog,shobj_description
3,pg_catalog,pg_typeof
4,pg_catalog,pg_typeof


In [6]:
%%sql
WHERE t.timestamp BETWEEN '{start_date}' AND '{end_date}'

(duckdb.CatalogException) Catalog Error: Table with name tweets does not exist!
Did you mean "information_schema.tables"?
LINE 2: FROM tweets 
             ^
[SQL: SELECT * 
FROM tweets 
WHERE content LIKE '%{keyword}%']
(Background on this error at: https://sqlalche.me/e/14/f405)


In [ ]:
def search_tweets(conn, search_type, keyword, start_date, end_date):
    cursor = conn.cursor()
    if search_type == "string":
        query = f'''SELECT * 
                    FROM tweets 
                    WHERE content LIKE '%{keyword}%' AND timestamp BETWEEN '{start_date}' AND '{end_date}' 
                    ORDER BY timestamp DESC;'''
    elif search_type == "hashtag":
        query = f'''SELECT t.* FROM tweets t
                    JOIN tweet_hashtags th ON t.tweet_id = th.tweet_id
                    JOIN hashtags h ON th.hashtag_id = h.hashtag_id
                    WHERE h.hashtag_text = '{keyword}' AND t.timestamp BETWEEN '{start_date}' AND '{end_date}'
                    ORDER BY t.timestamp DESC;'''
    elif search_type == "user":
        query = f"SELECT * FROM tweets WHERE user_id = {keyword} AND timestamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY timestamp DESC;"
    cursor.execute(query)
    return cursor.fetchall()

In [ ]:
def test_query_performance(conn,sqlstrings):
    test_queries = sqlstrings
    for query in test_queries:
        start_time = time.time()
        cursor = conn.cursor()
        cursor.execute(query)
        cursor.fetchall()
        end_time = time.time()
        print(f"Query:{query} took {end_time - start_time:.4f} seconds")